## Предметная область: Игра Dota 2

Dota 2 - многопользовательская компьютерная игра. В ней встречаются две команды по 5 человек в каждой, целью которой является уничтожение базы протиника.

В данной игре существует множество игровых режимов. В рамках проекта мы будем рассматривать режим Captain's Mode, который является основным режимом киберспортивных соревнований


## Как протекает матч

Каждая команда запрещает противнику и выбирает себе одних из более ста уникальных персонажей. Один игрок будет управлять своим героем на протяжении всего матча. Замену героев произвести нельзя

В течении матча игроки зарабатывают золото и опыт. Опыт позволяет повышать свой уровень и изучать новые способности. Золото дает возможность покупать предметы, которые так же усиляют персонажа.

Так же персонаж может погибнуть, после чего он выбывает из игры на определенное время, оставив команду в меньшенстве.

Игра заканчивается, когда одна из команд сносит главное здание противника "Трон"

## Постановка задачи

Предсказание победителя по данным о первых 5ти минутах игры

## Набор данных

Набор данных с матчами записан в файле matches.jsonlines.bz2. В каталоге dictionaries приведены расшифровки идентификаторов, которые присутствуют в записях матчей.

## Чтение данных

Информация о матчах записана в сжатом текстовом файле matches.jsonlines.bz2, каждая строчка которого содержит объект в формате JSON. Запись в формате JSON преобразуется в python-объект при помощи стандартного модуля json. Пример чтения матчей:


In [2]:
import json
import bz2

with bz2.BZ2File('./data/matches.jsonlines.bz2') as matches_file:
    for line in matches_file:
        match = json.loads(line)
        # Обработка матча
        break

#### Описание полей в записи матча

```python
{
    "match_id": 247,            # идентификатор матча
    "start_time": 1430514316,   # дата/время начала матча, unixtime
    "lobby_type": 0,            # тип комнаты, в которой собираются игроки 
                                #   (расшифровка в dictionaries/lobbies.csv)
 
    # стадия выбора героев
    "picks_bans": [
        {
            "order": 0,       # порядковый номер действия
            "is_pick": false, # true если команда выбирает героя, false — если банит
            "team": 1,        # команда, совершающая действие (0 — Radiant, 1 — Dire)
            "hero_id": 95     # герой, связанный с действием 
                              #    (расшифровка в dictionaries/heroes.csv)
        }, 
        ...
    ],

    # информация про каждого игрока, список ровно из 10 элементов
    # игроки с индексами от 0 до 4 — из команды Radiant, от 5 до 9 — Dire
    "players": [ 
        { 
        
            # герой игрока (расшифровка в dictionaries/heroes.csv)
            "hero_id": 67,  

            # временные ряды (отсчеты указаны в поле "times")
            "xp_t": [0, 13, 115, 177, 335, ...],   # опыт
            "gold_t": [0, 99, 243, 343, 499, ...], # золото + стоимость всех купленных вещей (net worth)
            "lh_t": [0, 0, 2, 2, 2, ...],          # количество убитых юнитов (не героев) противника

            # список событий: улучшение способностей героя
            "ability_upgrades": [
                {
                    "time": 51,      # игровое время
                    "level": 1,      # уровень игрока, на котором произошло улучшение
                    "ability": 5334  # способность, которая была улучшена 
                                     # (расшифровка в dictionaries/abilities.csv)
                }, 
                ...
            ],

            # список событий: убийства
            "kills_log": [
                {
                    "time": 831,    # игровое время
                    "player": 7,    # индекс игрока, чей герой был убит 
                                    #   (не заполнено, если был убит не герой)
                    "unit": "npc_dota_hero_viper" # тип убитого юнита
                }, 
                ...
            ],

            # список событий: покупка предметов
            "purchase_log": [
                {
                    "time": -73,     # игровое время
                                     #   точка отсчета игрового времени (ноль) начинается через
                                     #   несколько минут после фактического начала матча, поэтому
                                     #   время некоторых событий может быть отрицательным
                    "item_id": 44    # купленный предмер (расшифровка в dictionaries/items.csv)
                }, 
                ...
            ]

            # список событий: выкуп героя из таверны
            "buyback_log": [
                {"time": 2507},
                ...
            ],

            # список событий: установка героем "наблюдателей", позволяющих команде 
            # следить за чатью игрового поля на некотором расстоянии от точки установки
            "obs_log": [
                {
                    "time": 1711,    # игровое время установки
                    "xy": [111, 130] # координаты игрового поля
                }, 
                ...
            ],
            "sen_log": [], # аналогично полю obs_log, другой тип "наблюдателя"

        },
        ...
    ],
    
    # отсчеты игрового времени, в которые вычисляются значения временных рядов
    "times": [0, 60, 120, 180, ...],

    # ключевые события игры
    "objectives": [
        {
            "time": 198,           # время события
            "type": "firstblood",  # тип события
            "player1": 6,          # параметры события, могут содержать
            "player2": 1           #   индексы игроков (player), 
                                   #   номер команды (team, 0 — Radiant, 1 — Dire)
        }, 
        {
            "time": 765, 
            "type": "tower_kill", 
            "player": 7, 
            "team": 1
        }, 
        ...
    ]
    
    # итог матча (отсутствует в тестовых матчах)
    "finish": {
        "duration": 2980,             # длительность в секундах
        "radiant_win": false,         # true, если победила команда Radiant
        "tower_status_radiant": 0,    # состояние башен у команд к концу игры
        "tower_status_dire": 1972,    #   (см. описание битовой маски)
        "barracks_status_dire": 63,   # состояние бараков у команд к концу игры
        "barracks_status_radiant": 0  #   (см. описание битовой маски)
    }
}
```

#### Описание полей состояния башен и бараков

Состояние башен к концу игры задается целым числом, закодировано в битах:

```
┌─┬─┬─┬─┬─────────────────────── Not used.
│ │ │ │ │ ┌───────────────────── Ancient Bottom
│ │ │ │ │ │ ┌─────────────────── Ancient Top
│ │ │ │ │ │ │ ┌───────────────── Bottom Tier 3
│ │ │ │ │ │ │ │ ┌─────────────── Bottom Tier 2
│ │ │ │ │ │ │ │ │ ┌───────────── Bottom Tier 1
│ │ │ │ │ │ │ │ │ │ ┌─────────── Middle Tier 3
│ │ │ │ │ │ │ │ │ │ │ ┌───────── Middle Tier 2
│ │ │ │ │ │ │ │ │ │ │ │ ┌─────── Middle Tier 1
│ │ │ │ │ │ │ │ │ │ │ │ │ ┌───── Top Tier 3
│ │ │ │ │ │ │ │ │ │ │ │ │ │ ┌─── Top Tier 2
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ ┌─ Top Tier 1
│ │ │ │ │ │ │ │ │ │ │ │ │ │ │ │
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
```

Состояние бараков к концу игры закодировано в битах целого числа:

```
┌─┬───────────── Not used.
│ │ ┌─────────── Bottom Ranged
│ │ │ ┌───────── Bottom Melee
│ │ │ │ ┌─────── Middle Ranged
│ │ │ │ │ ┌───── Middle Melee
│ │ │ │ │ │ ┌─── Top Ranged
│ │ │ │ │ │ │ ┌─ Top Melee
│ │ │ │ │ │ │ │
0 0 0 0 0 0 0 0
```

## Извлечение признаков

Скрипт extract_features.py производит извлечение признаков из известной информации о матче за первые 5 игровых минут, составляет из них таблицу. Таблица поможет вам быстрее сформировать матрицу объект-признак, вектор ответов и начать применять методы машинного обучения для решения поставленной задачи.

Признаки, представленные в таблице `features.csv`, по мнению экспертов в предметной области являются наиболее важными для решения задачи предсказания победы команды. Тем не менее, не обязательно использовать эти признаки в исходном виде для применения методов машинного обучения — вы можете сделать новые признаки из имеющихся. Более того, признаки в файле `features.csv` содержат не всю информацию, известную про матч за первые 5 игровых минут. Вы можете использовать скрипт `extract_features.py` как пример и добавлять свои признаки для улучшения качества предсказания.

## Чтение данных с признаками

In [9]:
import pandas as pd
features = pd.read_csv('./data/features.csv', index_col='match_id')

features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


#### Описание признаков в таблице

- `match_id`: идентификатор матча в наборе данных
- `start_time`: время начала матча (unixtime)
- `lobby_type`: тип комнаты, в которой собираются игроки (расшифровка в `dictionaries/lobbies.csv`)
- Наборы признаков для каждого игрока (игроки команды Radiant — префикс `rN`, Dire — `dN`):
    - `r1_hero`: герой игрока (расшифровка в dictionaries/heroes.csv)
    - `r1_level`: максимальный достигнутый уровень героя (за первые 5 игровых минут)
    - `r1_xp`: максимальный полученный опыт
    - `r1_gold`: достигнутая ценность героя
    - `r1_lh`: число убитых юнитов
    - `r1_kills`: число убитых игроков
    - `r1_deaths`: число смертей героя
    - `r1_items`: число купленных предметов
- Признаки события "первая кровь" (first blood). Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение
    - `first_blood_time`: игровое время первой крови
    - `first_blood_team`: команда, совершившая первую кровь (0 — Radiant, 1 — Dire)
    - `first_blood_player1`: игрок, причастный к событию
    - `first_blood_player2`: второй игрок, причастный к событию
- Признаки для каждой команды (префиксы `radiant_` и `dire_`)
    - `radiant_bottle_time`: время первого приобретения командой предмета "bottle"
    - `radiant_courier_time`: время приобретения предмета "courier" 
    - `radiant_flying_courier_time`: время приобретения предмета "flying_courier" 
    - `radiant_tpscroll_count`: число предметов "tpscroll" за первые 5 минут
    - `radiant_boots_count`: число предметов "boots"
    - `radiant_ward_observer_count`: число предметов "ward_observer"
    - `radiant_ward_sentry_count`: число предметов "ward_sentry"
    - `radiant_first_ward_time`: время установки командой первого "наблюдателя", т.е. предмета, который позволяет видеть часть игрового поля
- Итог матча (данные поля отсутствуют в тестовой выборке, поскольку содержат информацию, выходящую за пределы первых 5 минут матча)
    - `duration`: длительность
    - `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    - Состояние башен и барраков к концу матча (см. описание полей набора данных)
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`

## Метрики качества

В качестве метрики качества будет использоваться площадь под ROC-кривой (AUC-ROC). В проекте мы будем использовать градиентный бустинг и логистическую регрессию, поэтому данная метрика качества подходит

## Предобработка данных

In [10]:
data_train=pd.read_csv('./features.csv',index_col='match_id')

data_train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [11]:
data_test=pd.read_csv('./features_test.csv',index_col='match_id')
data_test.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
6,1430287923,0,93,4,1103,1089,8,0,1,9,...,0,12.0,247.0,-86.0,272.0,3,4,2,0,118.0
7,1430293357,1,20,2,556,570,1,0,0,9,...,2,-29.0,168.0,-54.0,NaN,3,2,2,1,16.0
10,1430301774,1,112,2,751,808,1,0,0,13,...,1,-22.0,46.0,-87.0,186.0,1,3,3,0,-34.0
13,1430323933,1,27,3,708,903,1,1,1,11,...,2,-49.0,30.0,-89.0,210.0,3,4,2,1,-26.0
16,1430331112,1,39,4,1259,661,4,0,0,9,...,0,36.0,180.0,-86.0,180.0,1,3,2,1,-33.0


## Проверим пропуски данных в датасете

In [12]:
import numpy as np

In [13]:
for col in data_train.columns:
    pct_missing = np.mean(data_train[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))


start_time - 0%
lobby_type - 0%
r1_hero - 0%
r1_level - 0%
r1_xp - 0%
r1_gold - 0%
r1_lh - 0%
r1_kills - 0%
r1_deaths - 0%
r1_items - 0%
r2_hero - 0%
r2_level - 0%
r2_xp - 0%
r2_gold - 0%
r2_lh - 0%
r2_kills - 0%
r2_deaths - 0%
r2_items - 0%
r3_hero - 0%
r3_level - 0%
r3_xp - 0%
r3_gold - 0%
r3_lh - 0%
r3_kills - 0%
r3_deaths - 0%
r3_items - 0%
r4_hero - 0%
r4_level - 0%
r4_xp - 0%
r4_gold - 0%
r4_lh - 0%
r4_kills - 0%
r4_deaths - 0%
r4_items - 0%
r5_hero - 0%
r5_level - 0%
r5_xp - 0%
r5_gold - 0%
r5_lh - 0%
r5_kills - 0%
r5_deaths - 0%
r5_items - 0%
d1_hero - 0%
d1_level - 0%
d1_xp - 0%
d1_gold - 0%
d1_lh - 0%
d1_kills - 0%
d1_deaths - 0%
d1_items - 0%
d2_hero - 0%
d2_level - 0%
d2_xp - 0%
d2_gold - 0%
d2_lh - 0%
d2_kills - 0%
d2_deaths - 0%
d2_items - 0%
d3_hero - 0%
d3_level - 0%
d3_xp - 0%
d3_gold - 0%
d3_lh - 0%
d3_kills - 0%
d3_deaths - 0%
d3_items - 0%
d4_hero - 0%
d4_level - 0%
d4_xp - 0%
d4_gold - 0%
d4_lh - 0%
d4_kills - 0%
d4_deaths - 0%
d4_items - 0%
d5_hero - 0%
d5_level -

## Анализ пропусков
Процент отсутствия данных:

first_blood_time - 20%

first_blood_team - 20%

first_blood_player1 - 20%

first_blood_player2 - 45%

radiant_bottle_time - 16%

radiant_courier_time - 1%

radiant_flying_courier_time - 28%

radiant_first_ward_time - 2%

dire_bottle_time - 17%

dire_courier_time - 1%

dire_flying_courier_time - 27%

dire_first_ward_time - 2%

## Причина отсутствия данных:

параметры (first_blood_time first_blood_team first_blood_player1 first_blood_player2 ) вероятно пропущены по причинам того, что такие события не произошли, то есть в причине убийства персонажей не замешаны герои противника

параметры (radiant_bottle_time radiant_courier_time radiant_flying_courier_time radiant_first_ward_time dire_bottle_time dire_courier_time dire_flying_courier_time dire_first_ward_time) пропущены, так как эти предметы не были куплены

## Заполним пустые данные

In [14]:
data_train=data_train.fillna(-1)
data_test=data_test.fillna(-1)

Столбцом, содержащим целевую переменную является "Radiant_win"

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import time
import datetime

In [16]:
Y_train=data_train['radiant_win']
X_train = data_train.drop(['duration','radiant_win','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire'], 1)
kFold=KFold(shuffle=True,random_state=42,n_splits=5)


## 1: Градиентный бустинг

Обучим модель, применив градиентный бустинг. Найдем взаимосвязь между кол-вом деревьев, качеством модели и временем выполнения программного кода

In [43]:
def grad_boost(X,y,kf):
    treeArray=[10,20,30,40,50,60]
    #treeArray=[30]
    for k in treeArray:
        start_time = datetime.datetime.now()
        gb=GradientBoostingClassifier(n_estimators=k)
        gb.fit(X, y)
        quality = cross_val_score(gb, X, y, cv=kf, scoring='roc_auc')  
        print('обучение с параметром k='+str(k))
        print ('Time elapsed:', datetime.datetime.now() - start_time)
        print(quality)
        

In [45]:
grad_boost(X_train,Y_train,kFold)


обучение с параметром k=10
Time elapsed: 0:00:39.560336
[0.66383799 0.66635457 0.66360048 0.66529818 0.66516222]
обучение с параметром k=20
Time elapsed: 0:01:19.746940
[0.68083889 0.68272733 0.67969876 0.6834932  0.6855512 ]
обучение с параметром k=30
Time elapsed: 0:01:59.763007
[0.68892093 0.68934663 0.68712298 0.69180598 0.69283583]
обучение с параметром k=40
Time elapsed: 0:02:41.092825
[0.69264125 0.69335305 0.69153074 0.69586466 0.69680392]
обучение с параметром k=50
Time elapsed: 0:03:17.341451
[0.69627399 0.69747879 0.69470891 0.69921915 0.69979097]
обучение с параметром k=60
Time elapsed: 0:03:56.722569
[0.69904111 0.69974831 0.69741008 0.70187297 0.70252411]


## Оценим качество модели



![boosting](./boosting_2.png)

Как видно, с увеличением кол-ва деревьев увеличивается как качество модели, так и время выполнения программы.
Однако видно, что рост качества не пропорционален с ростом времени выполнения программы

## 2: Линейная регрессия
Обучим модель, применив линейную регрессию

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


In [60]:
param_grid_C = {'C': np.linspace(0.001, 0.01, 25)}


In [61]:
print (param_grid_C)

{'C': array([0.001   , 0.001375, 0.00175 , 0.002125, 0.0025  , 0.002875,
       0.00325 , 0.003625, 0.004   , 0.004375, 0.00475 , 0.005125,
       0.0055  , 0.005875, 0.00625 , 0.006625, 0.007   , 0.007375,
       0.00775 , 0.008125, 0.0085  , 0.008875, 0.00925 , 0.009625,
       0.01    ])}


In [62]:
def get_logreg_score(X, y,):   

    logreg_grid = GridSearchCV(LogisticRegression(random_state=241), param_grid_C, cv=kFold, 
                         scoring='roc_auc')
    logreg_grid.fit(X, y)
    print("Best parameter ", logreg_grid.best_params_)
    print("Best score ", logreg_grid.best_score_)   
    
    start_time = datetime.datetime.now()
    logreg = LogisticRegression(random_state=241, **logreg_grid.best_params_)
    logreg.fit(X, y)
    score_logreg = np.mean(cross_val_score(logreg, X, y, scoring='roc_auc', cv=kFold))
    print ("Time elapsed = " ,  datetime.datetime.now() - start_time)


Проверим изменение качества модели до и после масштабирования признаков

In [63]:
get_logreg_score(X_train, Y_train)
from sklearn.preprocessing import StandardScaler
X_train_scaled = pd.DataFrame(data = StandardScaler().fit_transform(X_train))

get_logreg_score(X_train_scaled, Y_train)


Best parameter  {'C': 0.001}
Best score  0.513447617665222
Time elapsed =  0:00:01.225568
Best parameter  {'C': 0.005875000000000001}
Best score  0.7165647780732918
Time elapsed =  0:00:02.758566


Из примера выше видно, что при масштабировании признаков - качество модели увеличивается на 20%, что является заметным улучшением

## Заменим категориальные признаки
Измерим качество модели, заменив категориальные на числовые. Удалим признак lobby_type, так как тип лобби всегда будет неизменен.

In [60]:
categ_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
X_train_nocateg = X_train.drop(categ_features, axis = 1)
X_train_nocateg = X_train_nocateg.fillna(0)
X_train_nocateg_scaled = pd.DataFrame(data = StandardScaler().fit_transform(X_train_nocateg))
categ_features.remove('lobby_type')


N= 112
X_pick = np.zeros((data_train.shape[0], N))

for i, match_id in enumerate(data_train.index):
    for p in range(5):
        X_pick[i, data_train.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data_train.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_pick_df = pd.DataFrame(data = X_pick)


X_train_bag = X_train_nocateg_scaled.join(X_pick_df, rsuffix='_')
get_logreg_score(X_train_bag, Y_train)


Best parameter  {'C': 0.01}
Best score  0.7517386754874054
Time elapsed =  0:00:08.406471


Замечаем, что преобразование признаков значительно увеличивает качество модели (~4%)

## Сравним 2 модели
Оценка качества модели при использовании градиентного бустинга с кол-вом деревьев равным 60 - 0,700119316 при времени работы 4,5 минуты. 
В свою очередь auc_score логистической регрессии после преобразования категориальных признаков равен 0.7517386754874054 при затраченном времени в 8 сек
В данном примере использование логистической регрессии более предпочтительно чем использование градиентного бустинга

## Изменим входные данные

Проверим, как изменится качество модели, если делать предсказания по первым 10ти, 15ти, 20ти минутам


## 1: Градиентный бустинг
Исследуем, как изменяется качество модели, если брать первые 10, 15 и 20 минут

In [40]:
features_10 = pd.read_csv('./data/features_10.csv', index_col='match_id')
features_10.head()

data_train_10 = features_10.sample (frac= 0.8 ,random_state= 0 )
data_test_10 = features_10.drop (data_train_10.index )

data_train_10=data_train_10.fillna(-1)
data_test_10=data_test_10.fillna(-1)

In [41]:
Y_train_10=data_train_10['radiant_win']
X_train_10 = data_train_10.drop(['duration','radiant_win','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire'], 1)
kFold=KFold(shuffle=True,random_state=42,n_splits=5)

In [46]:
grad_boost(X_train_10,Y_train_10,kFold)


обучение с параметром k=10
Time elapsed: 0:00:36.129886
[0.73060418 0.72397686 0.72499735 0.72511183 0.72029917]
обучение с параметром k=20
Time elapsed: 0:01:11.457579
[0.74378334 0.7389092  0.74296838 0.74344192 0.73916349]
обучение с параметром k=30
Time elapsed: 0:01:46.817666
[0.75107799 0.74731518 0.75017466 0.74901886 0.74470138]
обучение с параметром k=40
Time elapsed: 0:02:22.249212
[0.75441405 0.75145451 0.75394712 0.75270385 0.74849226]
обучение с параметром k=50
Time elapsed: 0:02:57.073843
[0.75733015 0.75447974 0.75681398 0.75555636 0.75268313]
обучение с параметром k=60
Time elapsed: 0:03:32.100491
[0.76024591 0.7566623  0.75940071 0.75738946 0.75448442]


In [48]:
features_15 = pd.read_csv('./data/features_15.csv', index_col='match_id')
features_15.head()

data_train_15 = features_15.sample (frac= 0.8 ,random_state= 0 )
data_test_15 = features_15.drop (data_train_10.index )

data_train_15=data_train_15.fillna(-1)
data_test_15=data_test_15.fillna(-1)

In [49]:
Y_train_15=data_train_15['radiant_win']
X_train_15 = data_train_15.drop(['duration','radiant_win','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire'], 1)


In [50]:
grad_boost(X_train_15,Y_train_15,kFold)


обучение с параметром k=10
Time elapsed: 0:00:38.376592
[0.76162977 0.77059262 0.76795162 0.76820012 0.76223934]
обучение с параметром k=20
Time elapsed: 0:01:15.154858
[0.78722998 0.79328829 0.79025956 0.7926801  0.78715647]
обучение с параметром k=30
Time elapsed: 0:01:52.908845
[0.79604016 0.80104856 0.79855311 0.80140082 0.79469085]
обучение с параметром k=40
Time elapsed: 0:02:30.823583
[0.80151511 0.80555643 0.80387502 0.80516768 0.80006026]
обучение с параметром k=50
Time elapsed: 0:03:08.381083
[0.80528994 0.80814439 0.80691856 0.80887247 0.80369939]
обучение с параметром k=60
Time elapsed: 0:03:45.481353
[0.80793266 0.81041406 0.80989561 0.81125397 0.80641313]


In [51]:
features_20 = pd.read_csv('./data/features_15.csv', index_col='match_id')
features_20.head()

data_train_20 = features_20.sample (frac= 0.8 ,random_state= 0 )
data_test_20 = features_20.drop (data_train_10.index )

data_train_20=data_train_20.fillna(-1)
data_test_20=data_test_20.fillna(-1)

In [52]:
Y_train_20=data_train_20['radiant_win']
X_train_20 = data_train_20.drop(['duration','radiant_win','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire'], 1)


In [ ]:
grad_boost(X_train_20,Y_train_20,kFold)


обучение с параметром k=10
Time elapsed: 0:00:42.813366
[0.76162977 0.77059262 0.76795162 0.76820012 0.76223934]
обучение с параметром k=20
Time elapsed: 0:01:20.717043
[0.78722998 0.79328829 0.79025956 0.7926801  0.78715647]
обучение с параметром k=30
Time elapsed: 0:01:59.972301
[0.79604016 0.80104856 0.79855311 0.80140082 0.79469085]
обучение с параметром k=40
Time elapsed: 0:02:50.994572
[0.80151511 0.80555643 0.80387502 0.80516768 0.80006026]


## Результат.
Как видно, при увеличении первых минут для предсказания, увеличивается так же и точность модели. Это объясняется тем, что точность предсказания увеличивается, когда дано больше информации о происходящем матче

## 2: Логистическая регрессия
Так же проверим, как изменяется качество модели при использовании логистической регрессии

In [68]:
X_train_scaled_10 = pd.DataFrame(data = StandardScaler().fit_transform(X_train_10))

categ_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
X_train_nocateg_10 = X_train_10.drop(categ_features, axis = 1)
X_train_nocateg_10 = X_train_nocateg_10.fillna(-1)
X_train_nocateg_scaled_10 = pd.DataFrame(data = StandardScaler().fit_transform(X_train_nocateg_10))
categ_features.remove('lobby_type')


N= 112
X_pick_10 = np.zeros((data_train_10.shape[0], N))

for i, match_id in enumerate(data_train_10.index):
    for p in range(5):
        X_pick_10[i, data_train_10.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_10[i, data_train_10.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_pick_df_10 = pd.DataFrame(data = X_pick_10)


X_train_bag_10 = X_train_nocateg_scaled_10.join(X_pick_df_10, rsuffix='_')

get_logreg_score(X_train_bag_10, Y_train_10)

Best parameter  {'C': 0.01}
Best score  0.8036783662961247
Time elapsed =  0:00:06.970339


In [69]:
X_train_scaled_15 = pd.DataFrame(data = StandardScaler().fit_transform(X_train_15))

categ_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
X_train_nocateg_15 = X_train_15.drop(categ_features, axis = 1)
X_train_nocateg_15 = X_train_nocateg_15.fillna(-1)
X_train_nocateg_scaled_15 = pd.DataFrame(data = StandardScaler().fit_transform(X_train_nocateg_15))
categ_features.remove('lobby_type')


N= 112
X_pick_15 = np.zeros((data_train_15.shape[0], N))

for i, match_id in enumerate(data_train_15.index):
    for p in range(5):
        X_pick_15[i, data_train_15.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_15[i, data_train_15.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_pick_df_15 = pd.DataFrame(data = X_pick_15)


X_train_bag_15 = X_train_nocateg_scaled_15.join(X_pick_df_15, rsuffix='_')

get_logreg_score(X_train_bag_15, Y_train_15)

Best parameter  {'C': 0.01}
Best score  0.8485838148065199
Time elapsed =  0:00:06.728482


In [70]:
X_train_scaled_20 = pd.DataFrame(data = StandardScaler().fit_transform(X_train_20))

categ_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
X_train_nocateg_20 = X_train_20.drop(categ_features, axis = 1)
X_train_nocateg_20 = X_train_nocateg_20.fillna(-1)
X_train_nocateg_scaled_20 = pd.DataFrame(data = StandardScaler().fit_transform(X_train_nocateg_20))
categ_features.remove('lobby_type')


N= 112
X_pick_20 = np.zeros((data_train_20.shape[0], N))

for i, match_id in enumerate(data_train_20.index):
    for p in range(5):
        X_pick_20[i, data_train_20.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_20[i, data_train_20.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_pick_df_20 = pd.DataFrame(data = X_pick_20)


X_train_bag_20 = X_train_nocateg_scaled_20.join(X_pick_df_20, rsuffix='_')

get_logreg_score(X_train_bag_20, Y_train_20)

Best parameter  {'C': 0.01}
Best score  0.8485838148065199
Time elapsed =  0:00:07.200900


## Результат
При увеличении времени при предсказании, качество модели с ипользованием логистической регрессии так же увеличивается

## Итоги
В ходе проведенных исследований в рамках выпускного проекта были реализованы две модели предсказания результатов матчей по игре "Dota 2". Использовались два подхода: Градиентный бустинг и логистическая регрессия. Качество модели измерялось с помощью  площади под ROC-кривой (AUC-ROC). Было выявлено, что логистическая регрессия имеет более качественный результат при меньшем кол-ве времени работы. Так же было проанализирована зависимость качества моделей от начального времени, после которого предсказывался итог игры. Закономерно получено, что качество моделей увеличивается при увеличении времени предсказания матча.